In [1]:
from datetime import datetime
import os
from utils import utils
import pandas as pd

import torch
from transformers.trainer_utils import set_seed
from transformers import RobertaTokenizer
from transformers import AutoConfig, AutoModelWithHeads
from transformers.adapters.composition import Fuse
from transformers import TrainingArguments, EvalPrediction, AdapterTrainer
from utils.evaluation import compute_pearsonr


RANDOM_SEED = 42
set_seed(RANDOM_SEED)

dateTimeObj = datetime.now()
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
# emotional-reactions eval_f1_macro 87.23
emo_adapter_path = "./trained_adapters/emotional-reactions"

# explorations eval_f1_macro 85.92
exp_adapter_path = "./trained_adapters/explorations"

# interpretations eval_f1_macro 57.67
int_adapter_path = "./trained_adapters/interpretations"

In [3]:
approach='EpitomeFusion'

""" choose from the options: distress, empathy """
prediction_task='distress'
prediction_task='empathy'


""" set training output directory """
training_output_dir = f"./training_output/{approach}_{prediction_task}_{dateTimeObj.hour}{dateTimeObj.minute}-{dateTimeObj.day}-{dateTimeObj.month}"

# Data

In [4]:
""" load data """

train_data, val_data, test_data = utils.load_wassa_dataset()

In [5]:
""" get task labels """

train_labels = list(train_data[prediction_task].values)
val_labels = list(val_data[prediction_task].values)
test_labels = list(test_data[prediction_task].values)

In [6]:
""" Prepare dataset for training: feature encodings """

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

train_encodings = tokenizer(list(train_data['essay'].values), truncation=True, padding=True)
val_encodings = tokenizer(list(val_data['essay'].values), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data['essay'].values), truncation=True, padding=True)

In [7]:
""" setup torch dataset """

class WassaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = WassaDataset(train_encodings, train_labels)
val_dataset = WassaDataset(val_encodings, val_labels)
test_dataset = WassaDataset(test_encodings, test_labels)

# Train Adapter

In [8]:
""" init model """

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)
model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/home/nbalguri/.local/lib/python3.9/site-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/home/nbalguri/.local/lib/python3.9/site-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertF

In [9]:
""" load epitome empathy adapters """

er_adapter = model.load_adapter(emo_adapter_path, 
                                load_as=f'{prediction_task}-emotional-reactions')
ex_adapter = model.load_adapter(exp_adapter_path, 
                                load_as=f'{prediction_task}-explorations')
ip_adapter = model.load_adapter(int_adapter_path, 
                                load_as=f'{prediction_task}-interpretations')

In [10]:
""" adapter composition """

# fusion layer
model.add_adapter_fusion(Fuse(er_adapter, ex_adapter, ip_adapter))
model.set_active_adapters(Fuse(er_adapter, ex_adapter, ip_adapter))

# classification head for target task
model.add_classification_head(f"{approach}-{prediction_task}", num_labels=1)

adapter_setup = Fuse(er_adapter, ex_adapter, ip_adapter)
model.train_adapter_fusion(adapter_setup)

In [11]:
""" training arguments """
from transformers import TrainingArguments, EvalPrediction, AdapterTrainer

num_train_epochs=10
per_device_train_batch_size=8
per_device_eval_batch_size=8
metric_for_best_model='eval_pearsonr'
warmup_steps=1000
weight_decay=0.1
learning_rate=5e-05

training_args = TrainingArguments(
    output_dir=training_output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    logging_dir='./logs',
    logging_steps=50,
    eval_steps=50,
    save_steps=50,
    evaluation_strategy='steps',
    disable_tqdm=False,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    save_strategy='steps',
#     load_best_model_at_end=True,
    metric_for_best_model=metric_for_best_model,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
""" init adapter trainer """

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_pearsonr,
)

In [13]:
trainer.train()

/home/nbalguri/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1860
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2330
  Number of trainable parameters = 23622154


Step,Training Loss,Validation Loss,Pearsonr,Pearsonr Scipy,Pval
50,19.359300,15.811317,-0.246100,-0.246131,0.000043
100,14.497700,10.323250,-0.190600,-0.190591,0.001655
150,7.516900,3.776831,-0.022500,-0.022499,0.712855
200,3.780300,3.526766,0.230800,0.230770,0.000130
250,3.501100,3.422412,0.304100,0.304077,0.000000
300,3.306100,3.223464,0.407900,0.407943,0.000000
350,3.050200,3.004418,0.401100,0.401079,0.000000
400,3.078800,3.013187,0.412900,0.412864,0.000000
450,2.804900,2.884973,0.434000,0.433993,0.000000
500,2.749700,2.931206,0.426400,0.426416,0.000000


***** Running Evaluation *****
  Num examples = 270
  Batch size = 8
Saving model checkpoint to ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-50
Configuration saved in ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-50/empathy-emotional-reactions/adapter_config.json
Module weights saved in ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-50/empathy-emotional-reactions/pytorch_adapter.bin
Configuration saved in ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-50/empathy-emotional-reactions/head_config.json
Module weights saved in ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-50/empathy-emotional-reactions/pytorch_model_head.bin
Configuration saved in ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-50/empathy-explorations/adapter_config.json
Module weights saved in ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-50/empathy-explorations/pytorch_adapter.bin
Configuration saved in ./training_output/Epitome

TrainOutput(global_step=2330, training_loss=3.2676859794256514, metrics={'train_runtime': 16854.9211, 'train_samples_per_second': 1.104, 'train_steps_per_second': 0.138, 'total_flos': 2761147486188000.0, 'train_loss': 3.2676859794256514, 'epoch': 10.0})

# Evaluate

In [14]:
""" load best model """

# load each individual adapter
model.load_adapter(
    trainer.state.best_model_checkpoint + f'/{er_adapter}'
)
model.load_adapter(
    trainer.state.best_model_checkpoint + f'/{ex_adapter}'
)
model.load_adapter(
    trainer.state.best_model_checkpoint + f'/{ip_adapter}'
)

# load adapter fusion
model.load_adapter_fusion(
    trainer.state.best_model_checkpoint + f"/{er_adapter},{ex_adapter},{ip_adapter}"
)

# set active adapters
model.set_active_adapters(Fuse(er_adapter, ex_adapter, ip_adapter))

# load head
model.load_head(trainer.state.best_model_checkpoint + f'/{approach}-{prediction_task}')

Loading module configuration from ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-1800/empathy-emotional-reactions/adapter_config.json
Overwriting existing adapter 'empathy-emotional-reactions'.
Loading module weights from ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-1800/empathy-emotional-reactions/pytorch_adapter.bin
Loading module configuration from ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-1800/empathy-emotional-reactions/head_config.json
Overwriting existing head 'empathy-emotional-reactions'
Adding head 'empathy-emotional-reactions' with config {'head_type': 'classification', 'num_labels': 3, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'0': 0, '1': 1, '2': 2}, 'use_pooler': False, 'bias': True}.
Loading module weights from ./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-1800/empathy-emotional-reactions/pytorch_model_head.bin
Loading module configuration from ./training_output/EpitomeFusion_empathy_1014-7-4/chec

('./training_output/EpitomeFusion_empathy_1014-7-4/checkpoint-1800/EpitomeFusion-empathy',
 'EpitomeFusion-empathy')

In [15]:
""" output eval metrics from best model """

#trainer.model.cuda()
eval_output = trainer.evaluate()
eval_result = eval_output[metric_for_best_model]

pd.DataFrame({'metric':list(eval_output.keys()), 'value': list(eval_output.values())}, columns=['metric', 'value'])

***** Running Evaluation *****
  Num examples = 270
  Batch size = 8


,metric,value
0,eval_loss,2.803352e+00
1,eval_pearsonr,5.138000e-01
2,eval_pearsonr_scipy,5.138177e-01
3,eval_pval,1.359033e-19
4,eval_runtime,7.189480e+01
5,eval_samples_per_second,3.755000e+00
6,eval_steps_per_second,4.730000e-01
7,epoch,1.000000e+01


# Test predictions

In [16]:
""" make predictions on test dataset """

p = trainer.predict(test_dataset)
preds = p.predictions[:, 0]

***** Running Prediction *****
  Num examples = 525
  Batch size = 8
/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [17]:
""" save predictions """

pred_file = f"{approach}-{metric_for_best_model}-{round(eval_result * 100, 4)}_{dateTimeObj.hour}{dateTimeObj.minute}-{dateTimeObj.day}-{dateTimeObj.month}.tsv"
pred_path = f'./predictions/{prediction_task}/{pred_file}'
os.makedirs(f'./predictions/{prediction_task}/', exist_ok=True)
pd.Series(preds).to_csv(pred_path, sep='\t', header=False, index=False)
print("saved predictions to",pred_path)

saved predictions to ./predictions/empathy/EpitomeFusion-eval_pearsonr-50.78_1838-21-4.tsv


# Save model

In [18]:
""" set path for where to save the adapter """
adapter_save_path = f"./trained_adapters/{approach}-{prediction_task}"

""" save each individual adapter """
os.makedirs(adapter_save_path + f"/{er_adapter}", exist_ok=True)
trainer.model.save_adapter(adapter_save_path + f"/{er_adapter}", er_adapter)

os.makedirs(adapter_save_path + f"/{ex_adapter}", exist_ok=True)
trainer.model.save_adapter(adapter_save_path + f"/{ex_adapter}", ex_adapter)

os.makedirs(adapter_save_path + f"/{ip_adapter}", exist_ok=True)
trainer.model.save_adapter(adapter_save_path + f"/{ip_adapter}", ip_adapter)

""" save fusion """
model.save_adapter_fusion(adapter_save_path, f"{er_adapter},{ex_adapter},{ip_adapter}")

""" save head """
model.save_head(adapter_save_path, f"{approach}-{prediction_task}")

Configuration saved in ./trained_adapters/EpitomeFusion-empathy/empathy-emotional-reactions/adapter_config.json
Module weights saved in ./trained_adapters/EpitomeFusion-empathy/empathy-emotional-reactions/pytorch_adapter.bin
Configuration saved in ./trained_adapters/EpitomeFusion-empathy/empathy-emotional-reactions/head_config.json
Module weights saved in ./trained_adapters/EpitomeFusion-empathy/empathy-emotional-reactions/pytorch_model_head.bin
Configuration saved in ./trained_adapters/EpitomeFusion-empathy/empathy-explorations/adapter_config.json
Module weights saved in ./trained_adapters/EpitomeFusion-empathy/empathy-explorations/pytorch_adapter.bin
Configuration saved in ./trained_adapters/EpitomeFusion-empathy/empathy-explorations/head_config.json
Module weights saved in ./trained_adapters/EpitomeFusion-empathy/empathy-explorations/pytorch_model_head.bin
Configuration saved in ./trained_adapters/EpitomeFusion-empathy/empathy-interpretations/adapter_config.json
Module weights saved